Hash Attack lab, implementing SHA-1 and and experminetally validating the difficulty of hash attacks

In [1]:
import binascii
import random
from collections import Counter
import matplotlib.pyplot as plt
import statistics as s
import random
import string

In [2]:
def is_integer(n):
    try:
        float(n)
    except ValueError:
        return False
    else:
        return float(n).is_integer()

In [3]:
def Ch(x,y,z):
    x_and_y = x & y
    notx_and_z = ~x & z
    xored = x_and_y ^ notx_and_z
    return xored

In [4]:
def Parity(x,y,z):
    xored = x ^ y ^ z
    return xored

In [5]:
def Maj(x,y,z):
    x_and_y = x & y
    x_and_z = x & z
    y_and_z = y & z
    xored = x_and_y ^ x_and_z ^ y_and_z
    return xored

Function that calls the coresponding the above functions depedning on 

In [6]:
def ft(x,y,z,t):
    if t >= 0 and t <= 19:
        return Ch(x,y,z)
    if t >= 20 and t <= 39:
        return Parity(x,y,z)
    if t >= 40 and t <= 59:
        return Maj(x,y,z)
    if t >= 60 and t <= 79:
        return Parity(x,y,z)

In [7]:
def ROTL(n,x):
    a = x << n
    b = x>>(32-n)
    return (a|b)&0xffffffff

In [8]:
# SHA-1 Constants Kt
#        0-19       20-39      40-59      60-79   
Kt = [0x5a827999,0x6ed9eba1,0x8f1bbcdc,0xca62c1d6]
# do val//20 to get indexs if val = 29//20 = 1 

Preprocessing consists of three steps: padding the message,parsing the message
into message blocks,and setting the initial hash value

Padding the message, this is used to ensure the message is a multiple of 512 bits, 

In [9]:
def Pad(word):
    msg_size = len(word)*8
    k = msg_size%448
    padded_msg = ""
    
    if msg_size >= 448:
        k-= 64

    for block in word:
        for i in range(len(block)): 
            #adding the binary value of word to the padded msg 
            padded_msg += ''.join(f"{ord(block[i]):08b}")
            

    padded_msg += "1" 
    for i in range(448-k-1):
        padded_msg += "0" 
    #gettin the message size in binarry and adding zero's infront of it 
    size = bin(msg_size) 
    size = size[2:] 
    padded_msg +=size.zfill(64) 

    return padded_msg


Parsing the Message and its padding must be parsed into N m-bit blocks. 16 M blocks with 32 bit words

In [10]:
#16 blocks of 32 bit words 
def Parsing(word):
    M = []
    for i in range(16):
        block = " "
        for j in range(32):
            block+=(word[(i*32)+j])
        M.append(int(block,2))
    return M

Before hash computation begins for each of the secure hash algorithms, the initial hash value,H0 must be set. The size and number of words in H0 depends on the message digest size.

In [11]:
def H0():
    H = [0x67452301, 0xefcdab89, 0x98badcfe, 0x10325476, 0xc3d2e1f0]
    return H

SHA-1 Hash Computation, we will use the constatns and functions used above to now implement Hash Computation

In [12]:
#preparing the message schedule Wt
def Schedule_WT(M):
    W = [0] * 80
    for t in range(0,80): 
        if t < 16: 
            #break Mi ininto 16 words W0, W1 .. W15 
            W[t] = M[t]   
        if (t >= 16): 
            w = W[t-3] ^ W[t-8] ^ W[t-14] ^ W[t-16]
            W[t] = ROTL(1,w)
    return W

In [13]:
# #SHA 1 has a has value of five 32-bit words
def SHA1(word = "abc"):
    H = H0()
    padded_msg = Pad(word)
    N = len(padded_msg)//512 
    for i in range(1,N+1):
        #get the corressponding block
        M = Parsing(padded_msg[(i-1)*512:i*512])

        Wt = Schedule_WT(M)


        
        a = H[0]
        b = H[1]
        c = H[2]
        d = H[3]
        e = H[4]
            
        for t in range(80):
            T = (ROTL(5,a) + ft(b,c,d,t) + e + Kt[t//20] + Wt[t]) & 0xffffffff
            e = d
            d = c
            c = ROTL(30,b)
            b = a
            a = T


        

        H[0] = (H[0] + a)&0xffffffff
        H[1] = (H[1] + b)&0xffffffff
        H[2] = (H[2] + c)&0xffffffff
        H[3] = (H[3] + d)&0xffffffff
        H[4] = (H[4] + e)&0xffffffff

    string = f'{hex(H[0])[2:]}{hex(H[1])[2:]}{hex(H[2])[2:]}{hex(H[3])[2:]}{hex(H[4])[2:]}'
    return string

In [14]:
test1 = "This is a test of SHA-1."
test2 = "Kerckhoff's principle is the foundation on which modern cryptography is built."
test3 = "SHA-1 is no longer considered a secure hashing algorithm."
test4 = "SHA-2 or SHA-3 should be used in place of SHA-1."
test5 = "Never roll your own crypto!"

print(SHA1(test1))
print(SHA1(test2))
print(SHA1(test3))
print(SHA1(test4))
print(SHA1(test5))

8e35b47213acb9fa620d8e884d3f6338166f34d7
f801ea3e4c55ca85928bbf1bb24776d61e3fe09
a0773c12a8851bcf9697b57ce3e3b49436f02cfe
dd102182aabb5778e925eb2f536bab904b97c9b5
ae912752721c0f7b5857cc8c314fb9a3e94ca1c0


Part 2—Experimentally validate the difficulty of hash attacks

In [15]:
def Wraper(msg,n):
    msg_digest = SHA1(msg)
    return msg_digest[:n]

In [16]:
def attack(msg,n):
    Hash = []
    given_hash = Wraper(msg,n)
    Hash.append(given_hash)
    
    preimage = 0
    collision = 0
    collion_check = False
    while True:

        random_string = ""
        letter_size = n//8

        # if isinstance(int_or_float, float) == False:
        if is_integer(n/8) == False:
                letter_size+=1
        for size in range(letter_size):
                random_string += random.choice(string.ascii_letters)
        

        preimage+=1
        new_hash = Wraper(random_string,n)
        if Hash.count(new_hash) > 1 and collion_check == False:
            collision = preimage
            collion_check = True

        Hash.append(new_hash)

        if new_hash == given_hash:
            break

    return preimage, collision

In [17]:
def Sample(n):
    

    x = [i for i in range(50)]
    preimage = []
    collision = []
    for i in range(50):
        random_string = ""
        letter_size = n//8

        # if isinstance(int_or_float, float) == False:
        if is_integer(n/8) == False:
                letter_size+=1
        for size in range(letter_size):
                random_string += random.choice(string.ascii_letters)
        
        pre,col = attack(random_string,n)

        preimage.append(pre)
        collision.append(col)

    
    return preimage,collision

In [18]:
N = [8,10,12,14,16,18,20,22]
preimage = []
collision = []
y_avg1 =[]
y_avg2 = []

y_avg1.append(None)
y_avg2.append(None)
for n in N:
    print(n)
    a,b = Sample(n)
    preimage.append(a)
    collision.append(b)
    y_avg1.append(s.mean(a))
    y_avg2.append(s.mean(b))
    

fig = plt.figure(figsize =(10, 7))
ax = fig.add_subplot(111)

plt.boxplot(preimage)


plt.plot(y_avg1, linestyle = '-', marker = ' ')
plt.title('Pre-image attack results', fontsize = 20)
plt.ylabel("Number of attempts")
plt.xlabel("number of bits")
plt.xticks([1,2,3,4,5,6,7,8], N)
plt.savefig("preimage.jpg")

plt.clf()


fig = plt.figure(figsize =(10, 7))
ax = fig.add_subplot(111)

# Creating axes instance
plt.boxplot(collision)

plt.plot(y_avg2, linestyle = '-', marker = ' ')
plt.title('Collision attack results', fontsize = 20)
plt.ylabel("Number of attempts")
plt.xlabel("number of bits")
plt.xticks([1,2,3,4,5,6,7,8], N)
plt.savefig("collision.jpg")

plt.clf()

8
10
12
14
16
18
20
22


<Figure size 1000x700 with 0 Axes>

<Figure size 1000x700 with 0 Axes>

In [19]:
print(y_avg1)
print(y_avg2)

[None, 37.7, 2702.6, 3035.86, 2390.82, 2554.86, 152350.8, 126415.3, 144283.24]
[None, 13.72, 295.44, 285.32, 294.64, 282.34, 4000.7, 4549.54, 4445.88]
